In [ ]:
def calculate_supertrend(df,period=14, multiplier=3):

    df['high_low'] = df['high'] - df['low']
    df['high_close'] = np.abs(df['high'] - df['close'].shift(1))
    df['low_close'] = np.abs(df['low'] - df['close'].shift(1))
    df['tr'] = df[['high_low', 'high_close', 'low_close']].max(axis=1)
    df['atr'] = df['tr'].rolling(window=period).mean()

    df['basic_upperband'] = ((df['high'] + df['low']) / 2) + (multiplier * df['atr'])
    df['basic_lowerband'] = ((df['high'] + df['low']) / 2) - (multiplier * df['atr'])
    
    df['supertrend'] = 0
    df['in_uptrend'] = True 
    
    for i in range(1, len(df)):
        previous_close = df['close'].iloc[i-1]
        previous_supertrend = df['supertrend'].iloc[i-1] if i > 1 else df['basic_lowerband'].iloc[i-1]
        upperband = df['basic_upperband'].iloc[i]
        lowerband = df['basic_lowerband'].iloc[i]

        if df['in_uptrend'].iloc[i-1]:
            df['supertrend'].iloc[i] = max(lowerband, previous_supertrend) if df['close'].iloc[i] > lowerband else upperband
            df['in_uptrend'].iloc[i] = df['close'].iloc[i] > lowerband
        else:
            df['supertrend'].iloc[i] = min(upperband, previous_supertrend) if df['close'].iloc[i] < upperband else lowerband
            df['in_uptrend'].iloc[i] = df['close'].iloc[i] > upperband
    
    df['signal_st'] = 0
    df.loc[df['in_uptrend'] == True, 'signal_st'] = 1
    df.loc[df['in_uptrend'] == False, 'signal_st'] = -1
    
    return df